In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from preprocessing_utils import *

In [2]:
data_files = [
    "../datasets/data_A.4lep.root",
    "../datasets/data_B.4lep.root",
    "../datasets/data_C.4lep.root",
    "../datasets/data_D.4lep.root",
]

df_data = load_many_root_files(
    data_files,
    label="data",
    verbose=True
)

Found TTrees:
  mini;1
Found TTrees:
  mini;1
Found TTrees:
  mini;1
Found TTrees:
  mini;1


In [ ]:
signal_files = {
    "ggH": ["../datasets/mc_345060.ggH125_ZZ4lep.4lep.root"],
    "VBF": ["../datasets/mc_344235.VBFH125_ZZ4lep.4lep.root"],
    "WH":  ["../datasets/mc_341964.WH125_ZZ4lep.4lep.root"],
    "ZH":  ["../datasets/mc_341947.ZH125_ZZ4lep.4lep.root"],
}

df_signal = pd.concat(
    [
        load_many_root_files(files, label=proc)
        for proc, files in signal_files.items()
    ],
    ignore_index=True
)

KeyboardInterrupt: 

In [5]:
df_bkg = load_many_root_files(
    ["../datasets/mc_363490.llll.4lep.root"],
    label="ZZ"
)